In [1]:
import openai
from dotenv import load_dotenv, find_dotenv

load_dotenv(find_dotenv())
client = openai.Client()

In [2]:
import pandas as pd

dataset = pd.read_csv('full_netflix_dataset.csv')
dataset.head(3)

,url,title,type,genres,releaseYear,imdbId,imdbAverageRating,imdbNumVotes,availableCountries
0,https://www.netflix.com/title/60000724,Forrest Gump,Movie,"Drama, Romance",1994.0,tt0109830,8.8,2311997.0,"AD, AE, AG, AL, AO, AR, AT, AU, BA, BB, BE, BG..."
1,https://www.netflix.com/title/1154386,The Fifth Element,Movie,"Action, Adventure, Sci-Fi",1997.0,tt0119116,7.6,516291.0,"AT, CH, DE"
2,https://www.netflix.com/title/60031236,Kill Bill: Vol. 1,Movie,"Action, Crime, Thriller",2003.0,tt0266697,8.2,1219973.0,"AE, AL, AO, AT, AU, AZ, BG, BH, BY, CA, CI, CM..."


### Cria o arquivo para ser enviado

In [3]:
file = client.files.create(
    file    = open('full_netflix_dataset.csv', 'rb'),
    purpose = 'assistants'
)
print(file)

FileObject(id='file-qVAixgzYXD7My4pwiUZdZfSB', bytes=3462237, created_at=1730270099, filename='full_netflix_dataset.csv', object='file', purpose='assistants', status='processed', status_details=None)


### Cria o [assistant](https://platform.openai.com/docs/assistants/overview/agents)
- `tool_resources` recebe os ids dos arquivos

In [4]:
DataAssistant = client.beta.assistants.create(
    name = "Engenheiro de Dados",
    instructions = 'Você é um Engenheiro de Dados que trabalha para a Netflix. Você recebeu o arquivo .CSV full_netflix_dataset.csv com informações importantes sobre o catálogo da Netflix. Sua tarefa é analisar o arquivo e responder as perguntas sobre os dados.',
    tools = [{'type': 'code_interpreter'}],
    tool_resources = {'code_interpreter': 
                        {'file_ids': 
                            [file.id]
                        }
                     },
    model = 'gpt-4o-mini'
)

### Cria e adiciona a mensagem em myThread

In [5]:
myThread = client.beta.threads.create()
print(myThread, end='\n\n')

message = client.beta.threads.messages.create( # cria uma mensagem
    thread_id = myThread.id, # na thread especificada
    role = 'user',
    content = 'Gere um gráfico com o ano de lançamento dos filmes do catálogo da Netflix no eixo x e a quantidade de filmes lançados no eixo y. Considere apenas os filmes que foram lançados entre 2000 e 2024 e que estão disponíveis no Brasil.'
)
print(message)

Thread(id='thread_e3pRp0SuEidXflfJPSCDsNvb', created_at=1730270101, metadata={}, object='thread', tool_resources=ToolResources(code_interpreter=None, file_search=None))

Message(id='msg_5GThn163DKzQxjX1EWBeov2u', assistant_id=None, attachments=[], completed_at=None, content=[TextContentBlock(text=Text(annotations=[], value='Gere um gráfico com o ano de lançamento dos filmes do catálogo da Netflix no eixo x e a quantidade de filmes lançados no eixo y. Considere apenas os filmes que foram lançados entre 2000 e 2024 e que estão disponíveis no Brasil.'), type='text')], created_at=1730270101, incomplete_at=None, incomplete_details=None, metadata={}, object='thread.message', role='user', run_id=None, status=None, thread_id='thread_e3pRp0SuEidXflfJPSCDsNvb')


### Roda a thread no assistant

In [6]:
run = client.beta.threads.runs.create(
    thread_id    = myThread.id,
    assistant_id = DataAssistant.id,
    instructions = 'O nome do usuário é Davi Brilhante e ele está pensando em assinar o serviço de streaming da netflix.'
)
print(run)

Run(id='run_oIWeu3TSnk0du2YuSKbqwUkX', assistant_id='asst_eAnD9R2hUjONmvzMB678R4z9', cancelled_at=None, completed_at=None, created_at=1730270102, expires_at=1730270702, failed_at=None, incomplete_details=None, instructions='O nome do usuário é Davi Brilhante e ele está pensando em assinar o serviço de streaming da netflix.', last_error=None, max_completion_tokens=None, max_prompt_tokens=None, metadata={}, model='gpt-4o-mini', object='thread.run', parallel_tool_calls=True, required_action=None, response_format='auto', started_at=None, status='queued', thread_id='thread_e3pRp0SuEidXflfJPSCDsNvb', tool_choice='auto', tools=[CodeInterpreterTool(type='code_interpreter')], truncation_strategy=TruncationStrategy(type='auto', last_messages=None), usage=None, temperature=1.0, top_p=1.0, tool_resources={})


### Aguarda a run terminar
- [estados que uma run pode assumir](https://platform.openai.com/docs/assistants/deep-dive/run-lifecycle)

In [7]:
while run.status in ['queued', 'in_progress', 'cancelling']:
    run = client.beta.threads.runs.retrieve( # recupera os dados da run 
        thread_id = myThread.id, # na thread especificada
        run_id = run.id # especifica qual run recuperar os dados
    )
    print(run.status)


in_progress
in_progress
in_progress
in_progress
in_progress
in_progress
in_progress
in_progress
in_progress
in_progress
in_progress
in_progress
in_progress
in_progress
in_progress
in_progress
in_progress
in_progress
in_progress
in_progress
in_progress
in_progress
in_progress
in_progress
in_progress
in_progress
in_progress
in_progress
in_progress
in_progress
in_progress
in_progress
in_progress
in_progress
in_progress
in_progress
in_progress
in_progress
in_progress
in_progress
in_progress
in_progress
in_progress
in_progress
in_progress
in_progress
in_progress
in_progress
in_progress
in_progress
in_progress
in_progress
in_progress
completed


### Verifica a resposta

In [8]:
if run.status == 'completed':
    mensagens = client.beta.threads.messages.list(
        thread_id = myThread.id
    )
    print(mensagens)
else:
    print('Erro:', run.status)

SyncCursorPage[Message](data=[Message(id='msg_hHrynld2zgzuYVzfEpvaxiYy', assistant_id='asst_eAnD9R2hUjONmvzMB678R4z9', attachments=[], completed_at=None, content=[ImageFileContentBlock(image_file=ImageFile(file_id='file-g3fIrnuDVlp1OWcF1jXcSOJ1', detail=None), type='image_file'), TextContentBlock(text=Text(annotations=[], value='Aqui está o gráfico que mostra a quantidade de filmes lançados na Netflix entre 2000 e 2024, disponíveis no Brasil. \n\n- O eixo X representa os anos de lançamento, enquanto o eixo Y representa a quantidade de filmes lançados em cada ano. \n\nSe precisar de mais alguma informação ou análise, é só avisar!'), type='text')], created_at=1730270118, incomplete_at=None, incomplete_details=None, metadata={}, object='thread.message', role='assistant', run_id='run_oIWeu3TSnk0du2YuSKbqwUkX', status=None, thread_id='thread_e3pRp0SuEidXflfJPSCDsNvb'), Message(id='msg_09Kg9ydTzIvxSocHriCDTfhn', assistant_id='asst_eAnD9R2hUjONmvzMB678R4z9', attachments=[], completed_at=None,

### Analisando os passos do modelo

In [9]:
run_steps = client.beta.threads.runs.steps.list(
  thread_id = myThread.id,
  run_id = run.id
)

In [10]:
for step in run_steps.data[::-1]:
    print('\n=== Step:', step.step_details.type)


=== Step: message_creation

=== Step: tool_calls

=== Step: message_creation

=== Step: tool_calls

=== Step: message_creation


In [17]:
for step in run_steps.data[::-1]:
    print('\n=== Step:', step.step_details.type)
    
    if step.step_details.type == 'tool_calls':
            for tool_call in step.step_details.tool_calls:
                print('-----')
                print(tool_call.code_interpreter.input)
                print('-----')
                # print('Result')
                # print(tool_call.code_interpreter.outputs[0].logs)
                
    if step.step_details.type == 'message_creation':
        message = client.beta.threads.messages.retrieve(
            thread_id  = myThread.id,
            message_id = step.step_details.message_creation.message_id
        )
        if message.content[0].type == 'text':
            print(message.content[0].text.value)
        elif message.content[0].type == 'image_file':
            file_id    = message.content[0].image_file.file_id
            image_data = client.files.content(file_id)
            try:
                with open('graficoGerado.png', 'wb') as f:
                    f.write(image_data.read())
                    print('Imagem salva com sucesso!')
            except Exception as e:
                print('Erro ao salvar a imagem:', e)


=== Step: message_creation
Para gerar o gráfico que você solicitou, primeiro, vou processar o arquivo que você enviou. Vou verificar a estrutura dos dados para entender como extrair as informações sobre o ano de lançamento dos filmes disponíveis na Netflix. Vamos dar uma olhada no arquivo.

=== Step: tool_calls
-----
# Lendo o arquivo enviado para entender sua estrutura
import pandas as pd

# O arquivo é um CSV, então vamos carregá-lo
file_path = '/mnt/data/file-qVAixgzYXD7My4pwiUZdZfSB'
data = pd.read_csv(file_path)

# Mostrando as primeiras linhas do DataFrame e suas colunas
data.head(), data.columns
-----

=== Step: message_creation
O arquivo contém as seguintes colunas relevantes:

- **releaseYear**: O ano de lançamento do filme.
- **availableCountries**: Os países onde o filme está disponível.

Para gerar o gráfico que você solicitou, seguiremos os seguintes passos:

1. Filtrar os filmes cujo ano de lançamento está entre 2000 e 2024.
2. Checar se o Brasil está incluído na lista d